In [2]:
import numpy as np
import pymongo

In [7]:
# Define the range for each variable
min_iot_devices = 10
max_iot_devices = 1000

# Cloud only
# min_edge_devices = 0
# max_edge_devices = 0
# Edge only
min_edge_devices = 20
max_edge_devices = 50

min_fog_devices = 0
max_fog_devices = 0

# Cloud only
# min_cloud_devices = 1
# max_cloud_devices = 10
# Edge only
min_cloud_devices = 0
max_cloud_devices = 0

min_idle_power = 10
max_idle_power = 50

min_max_power = 100
max_max_power = 300

min_max_instructions = 1000000
max_max_instructions = 5000000

min_net_delay_edge = 10
max_net_delay_edge = 300

min_net_delay_fog = 200
max_net_delay_fog = 1000

min_net_delay_cloud = 500
max_net_delay_cloud = 5000

# Define the range for the number of guest machines per host machine
min_guest_per_host = 1
max_guest_per_host = 5 

# Define the number of configurations (n)
n = 100

In [8]:
client_url = "mongodb://localhost:27017/"
db_name = "simulations"
# collection_name = "networks_cloud_only"
collection_name = "networks_edge_only"

In [9]:
# Initialize a dictionary to store the generated configurations
configurations = []

# Connect to MongoDB
client = pymongo.MongoClient(client_url)
db = client[db_name]
collection = db[collection_name] 

# Generate n random configurations
for i in range(n):
    config = {
        '_id': f'config-{i + 1}',  # Unique configuration ID
        'iot_devices': np.random.randint(min_iot_devices, max_iot_devices + 1),
        'edge_devices': np.random.randint(min_edge_devices, max_edge_devices + 1),
        'fog_devices': np.random.randint(min_fog_devices, max_fog_devices + 1),
        'cloud_devices': np.random.randint(min_cloud_devices, max_cloud_devices + 1),
    }
    
    # Generate configurations for each layer based on the relative number of machines
    total_machines = (
        config['iot_devices'] + config['edge_devices'] +
        config['fog_devices'] + config['cloud_devices']
    )
    
    # Define the number of host machines for each layer
    num_host_edge = int(config['edge_devices'])
    num_host_fog = int(config['fog_devices'])
    num_host_cloud = int(config['cloud_devices'])
    
    # Generate configurations for guest machines within each host machine
    for layer, num_hosts in [('edge', num_host_edge), ('fog', num_host_fog), ('cloud', num_host_cloud)]:
        host_machines = []
        for _ in range(num_hosts):
            num_guest = np.random.randint(min_guest_per_host, max_guest_per_host + 1)
            guest_machines = []
            for _ in range(num_guest):
                # Generate separate values for net_delay_request and net_delay_response based on the layer
                net_delay_request = np.random.uniform(min_net_delay_edge, max_net_delay_edge) if layer == 'edge' else np.random.uniform(min_net_delay_fog, max_net_delay_fog) if layer == 'fog' else np.random.uniform(min_net_delay_cloud, max_net_delay_cloud)
                net_delay_response = np.random.uniform(min_net_delay_edge, max_net_delay_edge) if layer == 'edge' else np.random.uniform(min_net_delay_fog, max_net_delay_fog) if layer == 'fog' else np.random.uniform(min_net_delay_cloud, max_net_delay_cloud)

                guest_config = {
                    'idle_cpu_utilization': np.random.uniform(min_idle_power, max_idle_power),
                    'max_cpu_utilization': np.random.uniform(min_max_power, max_max_power),
                    'max_instructions_per_second': np.random.randint(min_max_instructions, max_max_instructions + 1),
                    'net_delay_request': net_delay_request,
                    'net_delay_response': net_delay_response,
                    'layer': layer
                }
                guest_machines.append(guest_config)
            host_machines.append({'guest_machines': guest_machines})
        config[layer] = host_machines

    # Insert the configuration into MongoDB
    collection.insert_one(config)
    configurations.append(config)

# Close the MongoDB connection
client.close()

In [10]:
from networks import NetworkConfigurationManager

network_config_manager = NetworkConfigurationManager(client_url, db_name, collection_name)